## Test Co-Design FPS
This file tests the average FPS of the IsaacLab envs. <br>
It tests both the FPS for classic RL training on the tasks as well as the Co-Design FPS,<br>
which may differ due to the reset times between loading unique morphologies.

In [1]:
import time
import subprocess
import os

from stable_baselines3 import PPO

In [2]:
# EA parameters
timesteps_per_generation=13e4
pop_size = 8
generations = 3
ENV_COUNT = 1024

def print_stats(start_time, end_time, timesteps):
    time_diff = end_time - start_time
    print(f"Time taken: {time_diff} seconds, average FPS: {timesteps/(time_diff*ENV_COUNT)}")

### IsaacLab Testing

#### Regular RL FPS

In [ ]:
start_time = time.time()
result = subprocess.run(["python", "sb3/train.py", "--task", "Grasp-Object", 
                         "--headless", "--num_envs", "1024", "--seed", "200",
                         "--max_steps", f"{int(timesteps_per_generation*generations*pop_size)}"], 
                        capture_output=True, text=True)
end_time = time.time()

# Find the latest model path
base_dir = "logs/sb3/Grasp-Object"
model_dir = os.listdir(base_dir)[-1]
model_path = os.path.join(base_dir, model_dir, "model.zip")
model = PPO.load(model_path)

print_stats(start_time, end_time, model.num_timesteps)

[INFO][AppLauncher]: Using device: cuda:0
[INFO][AppLauncher]: Loading experience file: C:\Users\Ink\Hub\Projects\IsaacEnv\IsaacLab\apps\isaaclab.python.headless.kit
[Warning] [simulation_app.simulation_app] Modules: ['omni.kit_app'] were loaded before SimulationApp was started and might not be loaded correctly.
[Warning] [simulation_app.simulation_app] Please check to make sure no extra omniverse or pxr modules are imported before the call to SimulationApp(...)
Loading user config located at: 'c:/users/ink/miniconda3/envs/isaacenv/lib/site-packages/omni/data/Kit/Isaac-Sim/4.5/user.config.json'
[Info] [carb] Logging to file: c:/users/ink/miniconda3/envs/isaacenv/lib/site-packages/omni/logs/Kit/Isaac-Sim/4.5/kit_20250902_104819.log
2025-09-02 09:48:19 s] [Warning] [omni.kit.app.plugin] No crash reporter present, dumps uploading isn't available.
2025-09-02 09:48:19 s] [Warning] [omni.ext.plugin] [ext: rendering_modes] Extensions config 'extension.toml' doesn't exist 'c:/users/ink/hub/pro

c:\Users\Ink\miniconda3\envs\IsaacEnv\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [ ]:
model.num_timesteps / 932


562.5407725321888

#### Co-Design FPS

In [ ]:
start_time = time.time()
result = subprocess.run(["python", "population_evolution.py", "--task", "Grasp-Object", 
                         "--headless", "--num_envs", "1024", "--seed", "200", 
                         "--generations", f"{int(generations)}", "--population_size", f"{int(pop_size)}", 
                         "--timesteps_per_agent", f"{int(timesteps_per_generation)}", "--mutation_rate", "0"],
                        capture_output=True, text=True)
end_time = time.time()

# Find the latest model path
base_dir = "logs/sb3/population_evolution/grasp_object"
model_dirs = os.listdir(base_dir)
model_paths = [os.path.join(base_dir, d, f"{d}_gen_10.zip") for d in model_dirs]
models_timesteps = sum([PPO.load(p).num_timesteps for p in model_paths])

print_stats(start_time, end_time)

AttributeError: module 'os' has no attribute 'join'

In [10]:
print(result.stderr, result.stdout)

c:\Users\Ink\miniconda3\envs\IsaacEnv\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
 [INFO][AppLauncher]: Using device: cuda:0
[INFO][AppLauncher]: Loading experience file: C:\Users\Ink\Hub\Projects\IsaacEnv\IsaacLab\apps\isaaclab.python.headless.kit
[Warning] [simulation_app.simulation_app] Modules: ['omni.kit_app'] were loaded before SimulationApp was started and might not be loaded correctly.
[Warning] [simulation_app.simulation_app] Please check to make sure no extra omniverse or pxr mod